In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import shared_functions_setup as setup
import scipy.interpolate
import scipy.integrate

In [2]:
survey = 'LSST_DESI'
if (survey == 'SDSS'):
    import params as pa
elif (survey == 'LSST_DESI'):
    import params_LSST_DESI as pa
else:
    print "We don't have support for that survey yet; exiting."
    exit()

In [3]:
# Get the luminosity function, as a function of z, for two different limiting magnitudes

def get_phi(z, lum_params, mlim, survey):
	
	""" This function outputs the Schechter luminosity function with parameters fit in Loveday 2012, following the same procedure as Krause et al. 2015, as a function of z and L 
	The output is L[z][l], list of vectors of luminosity values in z, different at each z due to the different lowe luminosity limit, and phi[z][l], a list of luminosity functions at these luminosity vectors, at each z
	lum_params are the parameters of the luminosity function that are different for different samples, e.g. red vs all. lumparams = [Mr_s, Q, alpha_lum, phi_0, P]
	Note that the luminosity function is output both normalized (for getting Ai and ah) and unnormalized (for the red fraction)."""
	
	if (survey == 'SDSS'):
		import params as pa
	elif (survey == 'LSST_DESI'):
		import params_LSST_DESI as pa
	else:
		print "We don't have support for that survey yet; exiting."
		exit()
		
	[Mr_s, Q, alpha_lum, phi_0, P ] = lum_params
	
	# Get the amplitude of the Schechter luminosity function as a function of redshift.
	phi_s = phi_0 * 10.**(0.4 * P * z)
	
	# Get M_* (magnitude), then convert to L_*
	Ms = Mr_s - Q * (z - 0.1)
	Ls = 10**(-0.4 * (Ms - pa.Mp))
	
	# Import the kcorr and ecorr correction from Poggianti (assumes elliptical galaxies)
	# No data for sources beyon z = 3, so we keep the same value at higher z as z=3
	(z_k, kcorr, x,x,x) = np.loadtxt('./txtfiles/kcorr.dat', unpack=True)
	(z_e, ecorr, x,x,x) = np.loadtxt('./txtfiles/ecorr.dat', unpack=True)
	kcorr_interp = scipy.interpolate.interp1d(z_k, kcorr)
	ecorr_interp = scipy.interpolate.interp1d(z_e, ecorr)
	kcorr = kcorr_interp(z)
	ecorr = ecorr_interp(z)
	
	# Get the absolute magnitude and luminosity corresponding to limiting apparent magntiude (as a function of z)
	dl = setup.com(z, survey) * (1. + z)
	Mlim = mlim - (5. * np.log10(dl) + 25. + kcorr + ecorr)
	Llim = 10.**(-0.4 * (Mlim-pa.Mp))
	
	# Get the luminosity vectos - there will be a list of these, one for each redshift, because the limiting values are z-dependent
	L = [0] * len(z)
	for zi in range(0,len(z)):
		L[zi] = scipy.logspace(np.log10(Llim[zi]), 2., 1000)
		
	# Now get phi(L,z), where this exists for each z because the lenghts of the L vectors are different.
	phi_func = [0]*len(z)
	for zi in range(0,len(z)):
		phi_func[zi]= np.zeros(len(L[zi]))
		for li in range(0,len(L[zi])):
			phi_func[zi][li] = phi_s[zi] * (L[zi][li] / Ls[zi]) ** (alpha_lum) * np.exp(- L[zi][li] / Ls[zi])
			
	# Get the normalization in L as a function of z
	
	norm= np.zeros(len(z))
	phi_func_normed = [0]*len(z)
	for zi in range(len(z)):
		phi_func_normed[zi] = np.zeros(len(L[zi]))
		norm[zi] = scipy.integrate.simps(phi_func[zi], L[zi])
		phi_func_normed[zi] = phi_func[zi] / norm[zi]

	return (L, phi_func_normed, phi_func)
	
def get_fred_ofz(z, survey):
	""" This function gets the red fraction as a function of the redshift, using the Schecter luminosity function as defined in get_phi"""

	if (survey == 'SDSS'):
		import params as pa
	elif (survey == 'LSST_DESI'):
		import params_LSST_DESI as pa
	else:
		print "We don't have support for that survey yet; exiting."
		exit()

	(L_red, nothing, phi_red) = get_phi(z, pa.lumparams_red, survey)
	(L_all, nothing, phi_all) = get_phi(z, pa.lumparams_all, survey)
	
	# Integrate out in luminosity (phi is already normalized in luminosity
	phi_red_ofz = np.zeros(len(z))
	phi_all_ofz = np.zeros(len(z))
	for zi in range(0,len(z)):
		phi_red_ofz[zi] = scipy.integrate.simps(phi_red[zi], L_red[zi])
		phi_all_ofz[zi] = scipy.integrate.simps(phi_all[zi], L_all[zi])
		
	fred_ofz = phi_red_ofz / phi_all_ofz

	return fred_ofz

In [4]:
zvec = np.linspace(pa.zsmin, pa.zsmax, 1000)

# Get the luminosity function, as a function of z, for two different limiting magnitudes
(L_24, nothing, phi_24) = get_phi(zvec, pa.lumparams_all, 24., survey)
(L_25pt3, nothing, phi_25pt3)= get_phi(zvec, pa.lumparams_all, 25.3, survey)

In [5]:
# Integrate both in luminosity
phi_24_ofz = np.zeros(len(zvec))
phi_25pt3_ofz = np.zeros(len(zvec))
for zi in range(0,len(zvec)):
    phi_24_ofz[zi] = scipy.integrate.simps(phi_24[zi], L_24[zi])
    phi_25pt3_ofz[zi] = scipy.integrate.simps(phi_25pt3[zi], L_25pt3[zi])
    

In [6]:
# Get the source redshift distribution 
zsrc, N_ofz_unnormed = setup.get_NofZ_unnormed(pa.dNdzpar_fid, pa.dNdztype, pa.zsmin, pa.zsmax, 1000)
norm = scipy.integrate.simps(N_ofz_unnormed, zsrc)
dNdz = N_ofz_unnormed / norm


In [7]:
neff_24 = pa.n_s * scipy.integrate.simps(dNdz * phi_24_ofz / phi_25pt3_ofz, zvec)

In [8]:
print "neff_24 =", neff_24

neff_24 = 13.0491596813
